In [13]:
import torch
import pandas as pd
from transformers import pipeline, Conversation


In [14]:
chat_model = pipeline(task = "conversational", model = "facebook/blenderbot-400M-distill")

In [15]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import numpy as np
from helper import Transformer_Dataset, preprocess


In [16]:
model_path = "../Trained_Model/my_fine_tuned_model"

model = AutoModelForSequenceClassification.from_pretrained(model_path)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [17]:
tokenizer = AutoTokenizer.from_pretrained(model_path)

In [18]:
IElabel = {0 : "I", 1 : "E"}

NSlabel = {0 : "N", 1 : "S"}

FTlabel = {0 : "F", 1 : "T"}

JPlabel = {0 : "J", 1: "P"}


In [34]:
user_inputs = []
template = [{"role": "system", "content" : "You must ask about something related on user's personality."},
            ]


while 1:
    user_input = input()
    if user_input.lower() == "done":
        break

    print("You: ", user_input, flush=True)
    if user_input.lower() == "what is my mbti?":
        tokened_input = tokenizer(". ".join(user_inputs), max_length = 512, padding='max_length')
        tokened_input = {key: torch.tensor(value).to(device) for key, value in tokened_input.items()}
        outputs = model(**tokened_input)
        labeled = torch.round(torch.sigmoid(outputs["logits"]))
        output_label = [IElabel[int(labeled[0][0])], NSlabel[int(labeled[0][1])], FTlabel[int(labeled[0][2])], JPlabel[int(labeled[0][3])]]
        print("Bot: ", "".join(output_label))
    
    else:
        user_inputs.append(user_input)
    
    template.append({"role":"user", "content": f"{user_input}"})

    if len(template) > 5: 
        template.pop(1)
        template.pop(1)
        template.pop(1)
    try:
        bot_response = chat_model(template, max_length = 128)[-1]["content"]
        
        print("Bot: ", bot_response, flush = True)
        template.append({"role": "assistant", "content": f"{bot_response}"})
    except:
        template.pop(-1)
        print("You typed too many words, please make your sentence shorter.")


You:  hi
Bot:   Hi! How are you doing today? I'm doing well, thank you for asking. 
You:  what's up
Bot:   Not much, just hanging out at home. How about you? What are you up to?
You:  not so much
Bot:   What do you like to do for fun? I like to play video games and listen to music.
You:  okay
Bot:   What kind of music do you listen to? I listen to a lot of rock and roll.
